# Interactive Thematic Map of Ukraine Using <code>folium.Choropleth</code> Visualizing Method

Author: **Marcin Sikorski**<br>
Date: May, 2022

**Case Study**<br>
In this demonstration, we will visualize percentage of people with Ukrainian as their native language according to 2001 census (by region/oblast). We will use <code>folium</code> to make an interactive geospatial data visualization. The data for the plot come from Wikipedia (https://en.wikipedia.org/wiki/Ukrainian_language#/media/File:Ukraine_census_2001_Ukrainian.svg).

### Part 1 - Preparing Data

In [1]:
from urllib.request import urlopen
import json

# geojson of Ukraine
url = 'https://raw.githubusercontent.com/wmgeolab/geoBoundaries/6b002b1eee2fd9599f1a3af8fe076d694e6decee/\
releaseData/gbOpen/UKR/ADM1/geoBoundaries-UKR-ADM1_simplified.geojson'

# loading geojson data from url
with urlopen(url) as response:
    ukraine = json.load(response)

# checking properties for any element
ukraine['features'][3]['properties']

{'shapeName': 'Zhytomyr Oblast',
 'Level': 'ADM1',
 'shapeISO': 'UA-18',
 'shapeID': 'UKR-ADM1-14850775B22680847',
 'shapeGroup': 'UKR',
 'shapeType': 'ADM1'}

In [2]:
import pandas as pd

# preparing data
def dataframe():
    # initialize lists
    iso_code = []
    oblast = []
    percentage = [73.2, 97.3, 97, 93, 92.3, 89, 84, 53.8, 30, 24.1, 50.2, 95.3, 97.8, 81, \
                  98.3, 75.6, 46.3, 69.2, 10.1, 94.8, 95.2, 92.5, 90, 67, 88.9, 72.1, 6.8]
    
    # number of features (oblasts) in geojson
    n = len(ukraine['features'])
    
    # extracting data from geojson
    for i in range(0, n):
        iso_code.append(ukraine['features'][i]['properties']['shapeISO'])  
        oblast.append(ukraine['features'][i]['properties']['shapeName'])
    
    # creating dataframe
    df = pd.DataFrame(list(zip(oblast, iso_code, percentage)))
    df.columns = ['oblast', 'iso_code', 'percent'] 
    return df

df = dataframe()
df.tail()

,oblast,iso_code,percent
22,Poltava Oblast,UA-53,90.0
23,Dnipropetrovsk Oblast,UA-12,67.0
24,Kirovohrad Oblast,UA-35,88.9
25,Kyiv,UA-30,72.1
26,Sevastopol,UA-40,6.8


### Part 2 - Generating Choropleth Map

In [3]:
# adding new properties to features in geojson
def new_properties():
    # prepare the customized text
    tooltip_perc = []
    tooltip_oblast = []
    
    # creating new lists
    for i in range(len(df)):
        tooltip_perc.append(df['percent'][i])
        tooltip_oblast.append(df['oblast'][i].replace(' Oblast', ''))
        
    # append new properties into feature       
    for i in range(len(tooltip_perc)):
        ukraine['features'][i]['properties']['percent'] = tooltip_perc[i]
    for i in range(len(tooltip_oblast)):
        ukraine['features'][i]['properties']['oblast'] = tooltip_oblast[i]

new_properties()
ukraine['features'][5]['properties']

{'shapeName': 'Chernihiv Oblast',
 'Level': 'ADM1',
 'shapeISO': 'UA-74',
 'shapeID': 'UKR-ADM1-14850775B12610052',
 'shapeGroup': 'UKR',
 'shapeType': 'ADM1',
 'percent': 89.0,
 'oblast': 'Chernihiv'}

In [4]:
import folium

# 1. base map
map = folium.Map(location=[48.50, 31.60], zoom_start=6, tiles='openstreetmap', crs='EPSG3857')
    
custom_scale = (df['percent'].quantile((0, 0.1, 0.4, 0.6, 0.8, 1))).tolist()
    
# 2. layer with choropleth
choropleth = folium.Choropleth(
    geo_data = ukraine,
    name = 'Oblasts',
    data = df,
    columns = ['iso_code', 'percent'],
    key_on = 'feature.properties.shapeISO',
    threshold_scale = custom_scale,
    fill_color = 'YlOrRd',
    fill_opacity = 0.8,
    line_opacity = 0.2,
    line_color = 'black',
    line_weight = 0.7,
    nan_fill_color = 'white',
    nan_fill_opacity = 0.8,
    legend_name = 'Ukrainian as Primary Language in 2001',
    highlight = True,
    show = True
).add_to(map)
    
style_font = ('background-color: white; color: #333333; font-family: arial; font-size: 14px; padding: 10px;')
    
# 4. hovering tooltips
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ['oblast', 'percent'],
                                   aliases = ('Oblast:', 'Ukrainian Speakers [%]:'),
                                   labels = True,
                                   style = style_font,
                                   sticky = True))
    
# 5. adding layer control
folium.LayerControl(position='topright', collapsed=False).add_to(map)
    
map

### Part 3 - Customizing Choropleth Map

In [5]:
import branca.colormap as cm

# customized colorbar/colormap
def custom_colormap():
    color_list = ['#ffd080', '#f4a77a', '#e18079', '#c35e7d', '#9b4283', '#6c2d83']
    quantiles_list = [0, 0.1, 0.4, 0.6, 0.8, 1]
    
    colormap = cm.LinearColormap(colors=color_list,\
                                 vmin=0, vmax=100).to_step(data=df['percent'], method='quant', quantiles=quantiles_list)
    colormap.caption = 'Ukrainian as Primary Language in 2001'
    return colormap

colormap = custom_colormap()

# preview
colormap

In [6]:
# 1. base map
map_customized = folium.Map(location=[48.50, 31.60], zoom_start=6, max_bounds=True,\
                 tiles='cartodbpositron', control_scale=True, crs='EPSG3857')
    
# 2. custom functions
highlight_function = lambda x: {'fillColor': '#58edbe',
                                'color': 'white',
                                'weight': 0.1,
                                'fillOpacity': 0.2}

style_function = lambda x: {'fillColor': colormap(x['properties']['percent']),
                            'color': 'black',
                            'weight': 0.5,
                            'fillOpacity': 0.7}

style_font = ('background-color: white; color: #333333; font-family: arial; font-size: 14px; padding: 10px;')
    
# 3. layer with choropleth
choropleth = folium.features.GeoJson(
    data = ukraine,
    style_function = style_function,
    highlight_function = highlight_function,
    name = 'Oblasts Layer',
    control = True,
    
    # displaying Labels
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['oblast', 'percent'],
        aliases = ['Oblast:', 'Ukrainian Speakers [%]:'],
        labels = True,
        style = style_font,
        sticky = True)).add_to(map_customized)
    
# 4. adding customized colormap
colormap.add_to(map_customized)
    
# 5. adding layer with tile
folium.TileLayer(tiles='Stamen Terrain', overlay=True, name='Terrain Layer', control=True).add_to(map_customized)
    
# 7. adding layer control
folium.LayerControl(position='topright', collapsed=True).add_to(map_customized)
    
map_customized